## Ejercicio de Limpieza de base de datos


In [ ]:
import pandas as pd
import numpy as np
import locale
from typing import List, Dict, Any

# --- 1. SETUP DE LIMPIEZA Y PARSEO ROBUSTO ---
DATE_FORMATS = ['%d/%m/%Y %H:%M', '%d/%m/%Y', '%d-%b-%y', '%d-%b-%Y', '%b-%y', '%d/%m/%y', '%Y%m%d', '%m%d%y']
MONTH_MAP = {'ene': 'Jan', 'feb': 'Feb', 'mar': 'Mar', 'abr': 'Apr', 'may': 'May', 'jun': 'Jun', 'jul': 'Aug', 'ago': 'Aug', 'sep': 'Sep', 'oct': 'Oct', 'nov': 'Nov', 'dic': 'Dec'}

def replace_spanish_months(series: pd.Series) -> pd.Series:
    s = series.astype(str).str.lower()
    for spa, eng in MONTH_MAP.items():
        s = s.str.replace(spa, eng, regex=False)
    return s

def parsear_formatos(date_series: pd.Series, formats: List[str], dayfirst: bool = False) -> pd.Series:
    current_locale = locale.setlocale(locale.LC_TIME)
    parsed_series = pd.Series(pd.NaT, index=date_series.index)
    date_series = date_series.astype(str).str.strip()
    try:
        try:
            locale.setlocale(locale.LC_TIME, 'C') 
        except locale.Error:
            pass
        parsed_series = pd.to_datetime(date_series, errors='coerce', dayfirst=dayfirst)
        for fmt in formats:
            mask = parsed_series.isna()
            try:
                temp_parsed = pd.to_datetime(date_series[mask], format=fmt, errors='coerce')
                parsed_series.update(temp_parsed)
            except ValueError:
                continue
    finally:
        try:
            locale.setlocale(locale.LC_TIME, current_locale)
        except locale.Error:
            pass
    return parsed_series

def clean_numeric(series: pd.Series) -> pd.Series:
    if series.dtype == 'object':
        return series.astype(str).str.replace('.', '', regex=False).str.replace(',', '.', regex=False)
    return series

# --- 2. FUNCIONES DE VERIFICACIÓN (CON CORRECCIÓN N-3) ---

def verificar_conversion_fecha(df: pd.DataFrame, df_raw: pd.DataFrame, columna: str, indices: List[int]) -> pd.DataFrame:
    """Devuelve un DataFrame con los resultados de conversión para la columna."""
    max_index = df.index.max()
    # APLICACIÓN DE LA REGLA N-3: Los índices del documento se corrigen con -3
    indices_filtrados = [i - 3 for i in indices if i - 3 >= 0 and i - 3 <= max_index]
    resultados = []

    for i in indices_filtrados:
        original = df_raw.loc[i, columna]
        convertido = df.loc[i, columna]
        estado = "ÉXITO" if pd.notna(convertido) else "FALLO (NaT/Nulo)"
        
        resultados.append({
            'Fila (Doc)': i, # Usamos el número de fila del documento sin -1
            'Índice (Pandas)': i,
            'Valor Original': original,
            'Valor Depurado': convertido,
            'Estado': estado
        })
    return pd.DataFrame(resultados)

# --- 3. EJECUCIÓN DEL BLOQUE 1: ABIERTO ---

df = pd.read_csv("Datos.csv", sep=';')
df_raw = df.copy() 
indices_abierto = [151, 209, 284, 308, 338, 370, 250, 404, 353]

# Aplicar LIMPIEZA TOTAL
numeric_cols = ['Tiempo de trabajo', 'Horas snow', 'Horas Real', 'Horas Trabajo']
for col in numeric_cols:
    df[col] = clean_numeric(df[col]).astype(float)

date_cols = ['Abierto', 'Actualizado', 'Cerrado', 'Resuelto']
for col in date_cols:
    df[col] = replace_spanish_months(df[col]) 
    df[col] = parsear_formatos(df[col], formats=DATE_FORMATS, dayfirst=True)

# Generación de tabla
df_output = verificar_conversion_fecha(df, df_raw, 'Abierto', indices_abierto)
print("\n--- COLUMNA: ABIERTO (Casos de Cambio de Formato) ---")
print(df_output.to_markdown(index=False))


--- COLUMNA: ABIERTO (Casos de Cambio de Formato) ---
|   Fila (Doc) |   Índice (Pandas) | Valor Original   | Valor Depurado      | Estado   |
|-------------:|------------------:|:-----------------|:--------------------|:---------|
|          148 |               148 | 12/03/2024       | 2024-03-12 00:00:00 | ✅ ÉXITO |
|          206 |               206 | may-24           | 2024-05-01 00:00:00 | ✅ ÉXITO |
|          281 |               281 | 101424           | 2024-10-14 00:00:00 | ✅ ÉXITO |
|          305 |               305 | 112624           | 2024-11-26 00:00:00 | ✅ ÉXITO |
|          335 |               335 | 13/01/2025       | 2025-01-13 00:00:00 | ✅ ÉXITO |
|          367 |               367 | 24/02/2025       | 2025-02-24 00:00:00 | ✅ ÉXITO |
|          247 |               247 | 19/07/2024       | 2024-07-19 00:00:00 | ✅ ÉXITO |
|          401 |               401 | 30-abr-25        | 2025-04-30 00:00:00 | ✅ ÉXITO |
|          350 |               350 | 06-feb-25        | 2025-02

In [ ]:
import pandas as pd
import numpy as np
import locale
from typing import List, Dict, Any

# --- 1. SETUP DE LIMPIEZA Y PARSEO ROBUSTO (Idéntico al Bloque 1) ---

DATE_FORMATS = ['%d/%m/%Y %H:%M', '%d/%m/%Y', '%d-%b-%y', '%d-%b-%Y', '%b-%y', '%d/%m/%y', '%Y%m%d', '%m%d%y']
MONTH_MAP = {'ene': 'Jan', 'feb': 'Feb', 'mar': 'Mar', 'abr': 'Apr', 'may': 'May', 'jun': 'Jun', 'jul': 'Aug', 'ago': 'Aug', 'sep': 'Sep', 'oct': 'Oct', 'nov': 'Nov', 'dic': 'Dec'}

def replace_spanish_months(series: pd.Series) -> pd.Series:
    s = series.astype(str).str.lower()
    for spa, eng in MONTH_MAP.items():
        s = s.str.replace(spa, eng, regex=False)
    return s

def parsear_formatos(date_series: pd.Series, formats: List[str], dayfirst: bool = False) -> pd.Series:
    current_locale = locale.setlocale(locale.LC_TIME)
    parsed_series = pd.Series(pd.NaT, index=date_series.index)
    date_series = date_series.astype(str).str.strip()
    try:
        try:
            locale.setlocale(locale.LC_TIME, 'C') 
        except locale.Error:
            pass
        parsed_series = pd.to_datetime(date_series, errors='coerce', dayfirst=dayfirst)
        for fmt in formats:
            mask = parsed_series.isna()
            try:
                temp_parsed = pd.to_datetime(date_series[mask], format=fmt, errors='coerce')
                parsed_series.update(temp_parsed)
            except ValueError:
                continue
    finally:
        try:
            locale.setlocale(locale.LC_TIME, current_locale)
        except locale.Error:
            pass
    return parsed_series

def clean_numeric(series: pd.Series) -> pd.Series:
    if series.dtype == 'object':
        return series.astype(str).str.replace('.', '', regex=False).str.replace(',', '.', regex=False)
    return series

def verificar_conversion_fecha(df: pd.DataFrame, df_raw: pd.DataFrame, columna: str, indices: List[int]) -> pd.DataFrame:
    """Devuelve un DataFrame con los resultados de conversión para la columna."""
    max_index = df.index.max()
    # APLICACIÓN DE LA REGLA N-3: Los índices del documento se corrigen con -3
    indices_filtrados = [i - 3 for i in indices if i - 3 >= 0 and i - 3 <= max_index]
    resultados = []

    for i in indices_filtrados:
        original = df_raw.loc[i, columna]
        convertido = df.loc[i, columna]
        estado = "ÉXITO" if pd.notna(convertido) else "FALLO (NaT/Nulo)"
        
        resultados.append({
            'Fila (Doc)': i, # Usamos el número de fila del documento sin -1
            'Índice (Pandas)': i,
            'Valor Original': original,
            'Valor Depurado': convertido,
            'Estado': estado
        })
    return pd.DataFrame(resultados)

# --- 2. EJECUCIÓN DEL BLOQUE 2: ACTUALIZADO ---

df = pd.read_csv("Datos.csv", sep=';')
df_raw = df.copy() 
indices_actualizado = [91, 242, 392, 415, 398, 343, 294, 252, 230, 203, 174, 144]

# Aplicar LIMPIEZA TOTAL
numeric_cols = ['Tiempo de trabajo', 'Horas snow', 'Horas Real', 'Horas Trabajo']
for col in numeric_cols:
    df[col] = clean_numeric(df[col]).astype(float)
    if col == 'Horas snow':
        df['Horas snow original'] = df[col].copy()
df['Horas snow corregidas'] = df['Tiempo de trabajo'] / 3600

date_cols = ['Abierto', 'Actualizado', 'Cerrado', 'Resuelto']
for col in date_cols:
    df[col] = replace_spanish_months(df[col]) 
    df[col] = parsear_formatos(df[col], formats=DATE_FORMATS, dayfirst=True)

# Generación de tabla
df_output = verificar_conversion_fecha(df, df_raw, 'Actualizado', indices_actualizado)
print("\n--- COLUMNA: ACTUALIZADO (Casos de Cambio de Formato) ---")
print(df_output.to_markdown(index=False))


--- COLUMNA: ACTUALIZADO (Casos de Cambio de Formato) ---
|   Fila (Doc) |   Índice (Pandas) | Valor Original   | Valor Depurado      | Estado   |
|-------------:|------------------:|:-----------------|:--------------------|:---------|
|           88 |                88 | 121423           | 2023-12-14 00:00:00 | ✅ ÉXITO |
|          239 |               239 | 81224            | 2024-08-12 00:00:00 | ✅ ÉXITO |
|          389 |               389 | 4325             | 2025-04-03 00:00:00 | ✅ ÉXITO |
|          412 |               412 | 21-may-25        | 2025-05-21 00:00:00 | ✅ ÉXITO |
|          395 |               395 | 23-abr-25        | 2025-04-23 00:00:00 | ✅ ÉXITO |
|          340 |               340 | 29-ene-25        | 2025-01-29 00:00:00 | ✅ ÉXITO |
|          291 |               291 | 12-may-25        | 2025-05-12 00:00:00 | ✅ ÉXITO |
|          249 |               249 | 82624            | 2024-08-26 00:00:00 | ✅ ÉXITO |
|          227 |               227 | 62424            | 202

In [ ]:
import pandas as pd
import numpy as np
import locale
from typing import List, Dict, Any

# --- 1. SETUP DE LIMPIEZA Y PARSEO ROBUSTO ---

# NUEVA DEFINICIÓN: Incluye el formato de fecha natural
DATE_FORMATS_FULL = [
    '%d/%m/%Y %H:%M', '%d/%m/%Y', '%d-%b-%y', '%d-%b-%Y', 
    '%b-%y', '%d/%m/%y', '%Y%m%d', '%m%d%y',
    '%A, %d de %B de %Y' # <-- NUEVO FORMATO PARA LENGUAJE NATURAL
]

FULL_MAP = {
    'lunes,': 'Monday,', 'martes,': 'Tuesday,', 'miércoles,': 'Wednesday,', 'jueves,': 'Thursday,', 'viernes,': 'Friday,',
    'enero': 'January', 'febrero': 'February', 'marzo': 'March', 'abril': 'April', 'mayo': 'May', 'junio': 'June',
    'julio': 'July', 'agosto': 'August', 'septiembre': 'September', 'octubre': 'October', 'noviembre': 'November', 'diciembre': 'December',
    'ene': 'Jan', 'feb': 'Feb', 'mar': 'Mar', 'abr': 'Apr', 'may': 'May', 'jun': 'Jun',
    'jul': 'Jul', 'ago': 'Aug', 'sep': 'Sep', 'oct': 'Oct', 'nov': 'Nov', 'dic': 'Dec'
}

def replace_all_spanish_words(series: pd.Series) -> pd.Series:
    """Aplica el mapeo completo de abreviaturas y palabras a inglés."""
    s = series.astype(str).str.lower().str.strip()
    for spa, eng in FULL_MAP.items():
        s = s.str.replace(spa, eng, regex=False)
    return s

def parsear_formatos(date_series: pd.Series, formats: List[str], dayfirst: bool = False) -> pd.Series:
    current_locale = locale.setlocale(locale.LC_TIME)
    parsed_series = pd.Series(pd.NaT, index=date_series.index)
    date_series = date_series.astype(str).str.strip()
    try:
        try:
            locale.setlocale(locale.LC_TIME, 'C') 
        except locale.Error:
            pass
        parsed_series = pd.to_datetime(date_series, errors='coerce', dayfirst=dayfirst)
        for fmt in formats:
            mask = parsed_series.isna()
            try:
                temp_parsed = pd.to_datetime(date_series[mask], format=fmt, errors='coerce')
                parsed_series.update(temp_parsed)
            except ValueError:
                continue
    finally:
        try:
            locale.setlocale(locale.LC_TIME, current_locale)
        except locale.Error:
            pass
    return parsed_series

def clean_numeric(series: pd.Series) -> pd.Series:
    if series.dtype == 'object':
        return series.astype(str).str.replace('.', '', regex=False).str.replace(',', '.', regex=False)
    return series

def verificar_conversion_fecha(df: pd.DataFrame, df_raw: pd.DataFrame, columna: str, indices: List[int]) -> pd.DataFrame:
    """Devuelve un DataFrame con los resultados de conversión para la columna."""
    max_index = df.index.max()
    # APLICACIÓN DE LA REGLA N-3
    indices_filtrados = [i - 3 for i in indices if i - 3 >= 0 and i - 3 <= max_index]
    resultados = []

    for i in indices_filtrados:
        original = df_raw.loc[i, columna]
        convertido = df.loc[i, columna]
        estado = "ÉXITO" if pd.notna(convertido) else "FALLO (NaT/Nulo)"
        
        resultados.append({
            'Fila (Doc)': i, 
            'Índice (Pandas)': i,
            'Valor Original': original,
            'Valor Depurado': convertido,
            'Estado': estado
        })
    return pd.DataFrame(resultados)

# --- 2. EJECUCIÓN DEL BLOQUE 3: RESUELTO (CORRECCIÓN FINAL) ---

df = pd.read_csv("Datos.csv", sep=';')
df_raw = df.copy() 
indices_resuelto = [235, 242, 190]

# Aplicar LIMPIEZA TOTAL
numeric_cols = ['Tiempo de trabajo', 'Horas snow', 'Horas Real', 'Horas Trabajo']
for col in numeric_cols:
    df[col] = clean_numeric(df[col]).astype(float)

date_cols = ['Abierto', 'Actualizado', 'Cerrado', 'Resuelto']

# Aplicamos el mapeo a todas las columnas de fecha (incluyendo 'Resuelto')
for col in date_cols:
    df[col] = replace_all_spanish_words(df[col]) 
    # Usamos la lista de formatos extendida que incluye el formato natural
    df[col] = parsear_formatos(df[col], formats=DATE_FORMATS_FULL, dayfirst=True)

# Generación de tabla
df_output = verificar_conversion_fecha(df, df_raw, 'Resuelto', indices_resuelto)
print("\n--- COLUMNA: RESUELTO (Casos de Cambio de Formato) ---")
print(df_output.to_markdown(index=False))


--- COLUMNA: RESUELTO (Casos de Cambio de Formato) ---
|   Fila (Doc) |   Índice (Pandas) | Valor Original             | Valor Depurado      | Estado   |
|-------------:|------------------:|:---------------------------|:--------------------|:---------|
|          232 |               232 | 08/07/2024 8:43            | 2024-07-08 08:43:00 | ✅ ÉXITO |
|          239 |               239 | lunes, 5 de agosto de 2024 | 2024-08-05 00:00:00 | ✅ ÉXITO |
|          187 |               187 | lunes, 29 de abril de 2024 | 2024-04-29 00:00:00 | ✅ ÉXITO |


In [ ]:
import pandas as pd
import numpy as np
import locale
from typing import List, Dict, Any

# --- 1. SETUP DE LIMPIEZA Y PARSEO ROBUSTO (Idéntico al Bloque 1) ---

DATE_FORMATS = ['%d/%m/%Y %H:%M', '%d/%m/%Y', '%d-%b-%y', '%d-%b-%Y', '%b-%y', '%d/%m/%y', '%Y%m%d', '%m%d%y']
MONTH_MAP = {'ene': 'Jan', 'feb': 'Feb', 'mar': 'Mar', 'abr': 'Apr', 'may': 'May', 'jun': 'Jun', 'jul': 'Aug', 'ago': 'Aug', 'sep': 'Sep', 'oct': 'Oct', 'nov': 'Nov', 'dic': 'Dec'}

def replace_spanish_months(series: pd.Series) -> pd.Series:
    s = series.astype(str).str.lower()
    for spa, eng in MONTH_MAP.items():
        s = s.str.replace(spa, eng, regex=False)
    return s

def parsear_formatos(date_series: pd.Series, formats: List[str], dayfirst: bool = False) -> pd.Series:
    current_locale = locale.setlocale(locale.LC_TIME)
    parsed_series = pd.Series(pd.NaT, index=date_series.index)
    date_series = date_series.astype(str).str.strip()
    try:
        try:
            locale.setlocale(locale.LC_TIME, 'C') 
        except locale.Error:
            pass
        parsed_series = pd.to_datetime(date_series, errors='coerce', dayfirst=dayfirst)
        for fmt in formats:
            mask = parsed_series.isna()
            try:
                temp_parsed = pd.to_datetime(date_series[mask], format=fmt, errors='coerce')
                parsed_series.update(temp_parsed)
            except ValueError:
                continue
    finally:
        try:
            locale.setlocale(locale.LC_TIME, current_locale)
        except locale.Error:
            pass
    return parsed_series

def clean_numeric(series: pd.Series) -> pd.Series:
    if series.dtype == 'object':
        return series.astype(str).str.replace('.', '', regex=False).str.replace(',', '.', regex=False)
    return series

def verificar_correccion_horas(df: pd.DataFrame, columna: str, indices: List[int]) -> pd.DataFrame:
    """Devuelve un DataFrame con los resultados de corrección de Horas snow."""
    max_index = df.index.max()
    # APLICACIÓN DE LA REGLA N-3: Los índices del documento se corrigen con -3
    indices_filtrados = [i - 3 for i in indices if i - 3 >= 0 and i - 3 <= max_index]
    resultados = []
    
    for i in indices_filtrados:
        original = df.loc[i, 'Horas snow original']
        corregido = df.loc[i, 'Horas snow corregidas']
        diferencia = np.abs(original - corregido)
        
        estado = "CORREGIDO" if diferencia > 0.01 else "COINCIDE"
        
        resultados.append({
            'Fila (Doc)': i,
            'Índice (Pandas)': i,
            'Original (Horas)': original,
            'Corregido (Horas)': corregido,
            'Diferencia Abs': diferencia,
            'Estado': estado
        })
    return pd.DataFrame(resultados)

# --- 2. EJECUCIÓN DEL BLOQUE 4: HORAS SNOW ---

df = pd.read_csv("Datos.csv", sep=';')
df_raw = df.copy() 
indices_horas = [131, 162, 170, 207, 215, 238, 252, 272, 278, 289, 294, 305, 311, 327, 343, 366, 390, 408, 414]

# Aplicar LIMPIEZA TOTAL
numeric_cols = ['Tiempo de trabajo', 'Horas snow', 'Horas Real', 'Horas Trabajo']
for col in numeric_cols:
    df[col] = clean_numeric(df[col]).astype(float)
    if col == 'Horas snow':
        df['Horas snow original'] = df[col].copy()
df['Horas snow corregidas'] = df['Tiempo de trabajo'] / 3600

date_cols = ['Abierto', 'Actualizado', 'Cerrado', 'Resuelto']
for col in date_cols:
    df[col] = replace_spanish_months(df[col]) 
    df[col] = parsear_formatos(df[col], formats=DATE_FORMATS, dayfirst=True)

# Generación de tabla
df_output = verificar_correccion_horas(df, 'Horas snow', indices_horas)
print("\n--- COLUMNA: HORAS SNOW (Casos de Error de Cálculo) ---")
print(df_output.to_markdown(index=False, floatfmt=".4f"))


--- COLUMNA: HORAS SNOW (Casos de Error de Cálculo) ---
|   Fila (Doc) |   Índice (Pandas) |   Original (Horas) |   Corregido (Horas) |   Diferencia Abs | Estado       |
|-------------:|------------------:|-------------------:|--------------------:|-----------------:|:-------------|
|          128 |               128 |            12.4200 |              7.4167 |           5.0033 | ✅ CORREGIDO |
|          159 |               159 |             6.5000 |              3.5000 |           3.0000 | ✅ CORREGIDO |
|          167 |               167 |            22.8300 |             14.8333 |           7.9967 | ✅ CORREGIDO |
|          204 |               204 |             2.6700 |              1.6667 |           1.0033 | ✅ CORREGIDO |
|          212 |               212 |             3.1200 |              3.0000 |           0.1200 | ✅ CORREGIDO |
|          235 |               235 |             8.5000 |              8.0000 |           0.5000 | ✅ CORREGIDO |
|          249 |               249 | 

## Visualización del trabajo realizado

In [106]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import locale
from typing import List

# ==============================================================================
# 1. SETUP DE LIMPIEZA Y PARSEO ROBUSTO (MOTOR FINAL)
# ==============================================================================

DATE_FORMATS = [
    '%d/%m/%Y %H:%M', '%d/%m/%Y', '%d-%b-%y', '%d-%b-%Y', 
    '%b-%y', '%d/%m/%y', '%Y%m%d', '%m%d%y',  
]

MONTH_MAP = {
    'ene': 'Jan', 'feb': 'Feb', 'mar': 'Mar', 'abr': 'Apr', 'may': 'May', 'jun': 'Jun',
    'jul': 'Aug', 'ago': 'Aug', 'sep': 'Sep', 'oct': 'Oct', 'nov': 'Nov', 'dic': 'Dec'
}
# Agregamos los mapeos completos para manejar los strings largos
FULL_MAP = {
    'lunes,': 'Monday,', 'martes,': 'Tuesday,', 'miércoles,': 'Wednesday,', 'jueves,': 'Thursday,', 'viernes,': 'Friday,',
    'enero': 'January', 'febrero': 'February', 'marzo': 'March', 'abril': 'April', 'mayo': 'May', 'junio': 'June',
    'julio': 'July', 'agosto': 'August', 'septiembre': 'September', 'octubre': 'October', 'noviembre': 'November', 'diciembre': 'December',
}
# Unificamos todos los mapeos en una sola función
def replace_spanish_words(series: pd.Series) -> pd.Series:
    s = series.astype(str).str.lower().str.strip()
    full_map = {**MONTH_MAP, **FULL_MAP} # Combina abreviaturas y nombres completos
    for spa, eng in full_map.items():
        s = s.str.replace(spa, eng, regex=False)
    return s

def parsear_formatos(date_series: pd.Series, formats: List[str], dayfirst: bool = False) -> pd.Series:
    current_locale = locale.setlocale(locale.LC_TIME)
    parsed_series = pd.Series(pd.NaT, index=date_series.index)
    date_series = date_series.astype(str).str.strip()

    try:
        try:
            locale.setlocale(locale.LC_TIME, 'C') 
        except locale.Error:
            pass
            
        parsed_series = pd.to_datetime(date_series, errors='coerce', dayfirst=dayfirst)
        
        for fmt in formats:
            mask = parsed_series.isna()
            try:
                temp_parsed = pd.to_datetime(date_series[mask], format=fmt, errors='coerce')
                parsed_series.update(temp_parsed)
            except ValueError:
                continue

    finally:
        try:
            locale.setlocale(locale.LC_TIME, current_locale)
        except locale.Error:
            pass
            
    return parsed_series

def clean_numeric(series: pd.Series) -> pd.Series:
    if series.dtype == 'object':
        return series.astype(str).str.replace('.', '', regex=False).str.replace(',', '.', regex=False)
    return series

# ==============================================================================
# 2. FUNCIÓN PRINCIPAL DE EJECUCIÓN Y GENERACIÓN DE MÉTRICAS
# ==============================================================================

if __name__ == "__main__":
    
    # 0. Carga de datos
    try:
        df = pd.read_csv("Datos.csv", sep=';')
    except FileNotFoundError:
        print("Error: El archivo 'Datos.csv' no se encuentra.")
        sys.exit(1)
        
    # --- PREPARACIÓN DE COLUMNAS PARA EL ANÁLISIS DE CONTRASTE ---
    
    # 1. Limpieza Numérica y Creación de Columna Corregida
    numeric_cols = ['Tiempo de trabajo', 'Horas snow', 'Horas Real', 'Horas Trabajo']
    for col in numeric_cols:
        cleaned_values = clean_numeric(df[col])
        if col == 'Horas snow':
            # Guardamos el valor original (numérico) para el contraste
            df['Horas snow original'] = cleaned_values.astype(float)
        df[col] = cleaned_values.astype(float)
    df['Horas snow corregidas'] = df['Tiempo de trabajo'] / 3600

    # 2. Conversión de Fechas Robusta (para el cálculo de éxito)
    date_cols = ['Abierto', 'Actualizado', 'Cerrado', 'Resuelto']
    df_raw_dates = df[date_cols].copy() # Fechas en string para métrica de NaT inicial

    for col in date_cols:
        df[col] = replace_spanish_words(df[col]) 
        df[col] = parsear_formatos(df[col], formats=DATE_FORMATS, dayfirst=True)

    # ==========================================================================
    #             CÁLCULO DE MÉTRICAS DE IMPACTO
    # ==========================================================================
    
    # 1. Impacto en la Corrección de Horas
    total_filas = len(df)
    df['Diferencia Horas'] = np.abs(df['Horas snow original'] - df['Horas snow corregidas'])
    casos_corregidos = (df['Diferencia Horas'] > 0.01).sum() # Diferencia > 36 segundos
    
    # 2. Impacto en la Conversión de Fechas
    # Contamos el NaT original vs. el NaT después de la limpieza
    original_na = df_raw_dates.isnull().sum().sum()
    final_na = df[date_cols].isna().sum().sum()
    
    # Excluimos los NaT genuinos (los que no tienen string, ej. Resuelto)
    original_non_na_total = (df[date_cols].apply(lambda x: x.astype(str).str.strip() != 'nan')).sum().sum()
    casos_string_a_fecha = original_non_na_total - (df[date_cols].notna().sum().sum())
    
    print("=====================================================================")
    print("                MÉTRICAS DE IMPACTO DEL TRABAJO DEPURADO")
    print("=====================================================================")
    print(f"Total de registros analizados: {total_filas}")
    print("\n--- A. CORRECCIÓN NUMÉRICA (Horas snow) ---")
    print(f"Casos con error de cálculo corregido: {casos_corregidos} filas.")
    print(f"Porcentaje de impacto en la calidad: {casos_corregidos / total_filas * 100:.2f}%.")

    print("\n--- B. CONVERSIÓN DE FORMATO (Fechas) ---")
    print(f"Total de valores NaT en el CSV original: {original_na}")
    print(f"Total de valores NaT después de la limpieza robusta: {final_na}")
    print(f"Valores de texto transformados a fecha: {casos_string_a_fecha} casos.")

    # ==========================================================================
    #                 GENERACIÓN DE GRÁFICOS DE CONTRASTE
    # ==========================================================================
    sns.set_style("whitegrid")
    
    # 1. Gráfico: Distribución de Horas Snow (Original vs. Corregida)
    plt.figure(figsize=(12, 6))
    
    # Datos limpios y listos para plotear
    sns.histplot(df['Horas snow original'], bins=30, kde=True, color='red', label='Original (con errores)', alpha=0.6)
    sns.histplot(df['Horas snow corregidas'], bins=30, kde=True, color='green', label='Corregida (precisa)', alpha=0.6)
    
    plt.title('Distribución de Horas Snow: Original vs. Corregida', fontsize=14)
    plt.xlabel('Horas Snow', fontsize=12)
    plt.ylabel('Frecuencia', fontsize=12)
    plt.legend(title='Datos', fontsize=10)
    plt.xlim(0, df['Horas snow corregidas'].quantile(0.99)) # Limitar para visualización clara
    plt.tight_layout()
    plt.savefig('grafico_horas_snow_contraste.png')
    plt.close()
    
    print("\nSe generó 'grafico_horas_snow_contraste.png' para visualizar la corrección de outliers.")

    # 2. Gráfico: Resumen de Éxito de Conversión de Fechas (NaT antes y después)
    
    # Calcular NaT por columna antes y después
    df_na_contrast = pd.DataFrame({
        'Columna': date_cols,
        'NaT Inicial': df_raw_dates.isna().sum().values,
        'NaT Final (Corregido)': df[date_cols].isna().sum().values
    })
    df_na_contrast_melted = df_na_contrast.melt(id_vars='Columna', var_name='Estado', value_name='Conteo NaT/NaN')

    plt.figure(figsize=(10, 6))
    sns.barplot(x='Columna', y='Conteo NaT/NaN', hue='Estado', data=df_na_contrast_melted,
                palette={'NaT Inicial': 'skyblue', 'NaT Final (Corregido)': 'darkblue'})
    
    plt.title('Contraste de Calidad de Fechas: NaT/NaN Inicial vs. Final', fontsize=14)
    plt.xlabel('Columna de Fecha', fontsize=12)
    plt.ylabel('Conteo de Valores Faltantes (NaT/NaN)', fontsize=12)
    plt.tight_layout()
    plt.savefig('grafico_nat_contraste.png')
    plt.close()
    
    print("Se generó 'grafico_nat_contraste.png' para mostrar el éxito de la conversión de formatos.")

                MÉTRICAS DE IMPACTO DEL TRABAJO DEPURADO
Total de registros analizados: 414

--- A. CORRECCIÓN NUMÉRICA (Horas snow) ---
Casos con error de cálculo corregido: 19 filas.
Porcentaje de impacto en la calidad: 4.59%.

--- B. CONVERSIÓN DE FORMATO (Fechas) ---
Total de valores NaT en el CSV original: 175
Total de valores NaT después de la limpieza robusta: 185
Valores de texto transformados a fecha: 185 casos.

Se generó 'grafico_horas_snow_contraste.png' para visualizar la corrección de outliers.
Se generó 'grafico_nat_contraste.png' para mostrar el éxito de la conversión de formatos.
